# Water Column Modeling
> This notebook will perform our water column modeling for each entry of the vegetation density data, giving us flow properties (U, Q2, Kz etc..) for each vegetation density profile

Developed by Alexandre Erich Sebastien Georges, PhD Student in Environmental Engineering at UC Berkeley <br> Fall 2022

In [2]:
# Import water column model class and components
from water_column_model.column import Column
from water_column_model.advance import *
from water_column_model.params import *
from water_column_model.params import A,B,C,E
# Data Science and Visualization Imports
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import xarray as xr
import numpy as np
# Miscellaneous
from tqdm import tqdm

## Importing USGS Data

In [3]:
filename = 'data/USGS_veg_density_09.16.2022.csv'
df = pd.read_csv(filename)
df.head()

,Season,date,site,plot_ID,Elevation,channel_dist,ave_ht,cover,vol_est,vol_calc,density_naive,density_final,frontal_total,frontal_5,frontal_10,frontal_15,frontal_20,frontal_25,frontal_30
0,Summer,7/28/2021,Bay,A0.5,2.1313,0.5,6.5,0.10,6500,233.351057,267.040006,630.481094,1103.449997,848.807690,254.642307,0.000000,0.000000,0.0,0.0
1,Summer,7/28/2021,Bay,A2,2.3217,2.0,11.0,0.45,49500,1777.058049,2033.612355,2837.164924,8403.196133,3819.634606,3819.634606,763.926921,0.000000,0.0,0.0
2,Summer,7/28/2021,Bay,A6,2.2421,6.0,19.0,0.55,104500,3751.566993,4293.181638,3467.646018,17740.080726,4668.442296,4668.442296,4668.442296,3734.753837,0.0,0.0
3,Summer,7/28/2021,Bay,A12,2.2417,12.0,15.0,0.75,112500,4038.768294,4621.846261,4728.608206,19098.173030,6366.057677,6366.057677,6366.057677,0.000000,0.0,0.0
4,Summer,7/28/2021,Bay,A24,2.2217,24.0,11.0,0.55,60500,2171.959838,2485.526211,3467.646018,10270.573052,4668.442296,4668.442296,933.688459,0.000000,0.0,0.0


Temporary: Only using data from Bay Sites

In [4]:
df['ave_ht'].describe()

count    114.000000
mean      15.346491
std        5.261787
min        4.000000
25%       12.000000
50%       15.000000
75%       19.000000
max       30.000000
Name: ave_ht, dtype: float64

In [5]:
df = df[df['site'] == 'Bay']
df['site'].value_counts()

Bay    42
Name: site, dtype: int64

## Making Water Columns for Data Entries

In [6]:
# Setting max water column height (arbitrary for now, but based on maximum vegetation height from data)
H = 30

In [7]:
# for a given set of parameteres, create a column, run the model and return Flow Properties
def run_column_model(N, H, L, SMALL, params, veg_density, veg_ht):
    # Initializing Column Object
    col = Column(N, H, L, SMALL)
    A = params[0]
    B = params[1]
    C = params[2]
    Sq = params[3]
    kappa = params[4]
    nu = params[5]
    g = params[6]
    rho0 = params[7]
    alpha = params[8]
    # Setting up vlaues and importing Vegetation Distribution from Data
    col = col.setup(A, B, C, Sq, kappa, SMALL, nu, g, rho0, alpha)
    col = col.import_veg(veg_density, veg_ht)
    # Running Model for Col.M amount of time (See Column Class to change M)
    t = [] 
    for i in tqdm(range(col.M), leave=False):
        t.append(col.dt*(i))
        # Unew, Cnew, Qnew, Q2new, Q2Lnew, rhonew, Lnew, nu_tnew, Kznew, Kqnew, N_BVnew, N_BVsqnew
        [col.U, col.scalar, col.Q, col.Q2, col.Q2L, col.rho, col.L, col.nu_t, col.Kz, col.Kq, col.N_BV, col.N_BVsq] = wc_advance(col, t_px, px0, t[i])
        # Have TQDM instead print('Step! t=' +str(t[i])+'s')
    ### Only for now ### Return Velocity Profile ### Will implement returning more later
    col_res = pd.DataFrame({'U Velocity':col.U,
                            'Q':col.Q,
                            'Q2':col.Q2,
                            'Q2L':col.Q2L,
                            'Z':col.z,
                            'Kq':col.Kq,
                            'nu_t':col.nu_t,
                            })
    return col_res

In [8]:
# New dataframe without unneccessary columns
wcolumns = df.drop(['plot_ID', 'frontal_5', 'frontal_10', 'frontal_15', 'frontal_20', 'frontal_20', 'frontal_25', 'frontal_30'], axis=1)
# Pulling parameters
params = [A, B, C, Sq, kappa, nu, g, rho0, alpha]

# Initializing and running water column model for each entry
col_res = wcolumns.apply(lambda x: run_column_model(80,H,0,SMALL,params,x.density_final,x.ave_ht), axis=1)

Turning results into a Xarray for easy processing and visualization

In [68]:
#xr.Dataset(col_res.tolist())
wc_ds = xr.concat([df.to_xarray() for df in col_res], dim="Model Runs")
wc_ds = wc_ds.assign_attrs(description='Model run results for Bay-side sites.')

Compiling Data from modeling runs for analysis

In [126]:
df_res = pd.DataFrame({
    'mean_u': ([vel.values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_25': ([vel[:20].values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_50': ([vel[:40].values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_75': ([vel[:60].values.mean() for vel in wc_ds['U Velocity']]),
    'avg_ht': df['ave_ht'],
    'density_total': df['density_final'],
    'frontal_total': df['frontal_total'],
})
df_res.head()

,mean_u,mean_u_25,mean_u_50,mean_u_75,avg_ht,density_total,frontal_total
0,-0.435630,-0.012249,-0.164500,-0.312611,6.5,630.481094,1103.449997
1,-0.205024,-0.000594,-0.028733,-0.116369,11.0,2837.164924,8403.196133
2,-0.055379,-0.000537,-0.000537,-0.011121,19.0,3467.646018,17740.080726
3,-0.104845,-0.000460,-0.000536,-0.041661,15.0,4728.608206,19098.173030
4,-0.204767,-0.000537,-0.028582,-0.116149,11.0,3467.646018,10270.573052


# Visualization

In [10]:
px.line(x=col_res[0]['Q'], y=col_res[0]['Z'], title='Velocity Profile', labels={'x':'Velocity [m/s]', 'y':'Z [m]'})